## Sentiment Analysis

### Connect to Twitter / X

In [ ]:
import twitter

# keys redacted

api = twitter.Api(consumer_key=consumer_key,
                      consumer_secret=consumer_secret,
                      access_token_key=access_token_key,
                      access_token_secret=access_token_secret)

print(api.VerifyCredentials())

{"created_at": "Sun Jun 01 18:12:58 +0000 2025", "default_profile": true, "default_profile_image": true, "friends_count": 1, "id": 1929239737425965056, "id_str": "1929239737425965056", "name": "nyour", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "nyour52", "withheld_in_countries": []}


### Get stream of data (no longer available on free tier)

In [16]:
for tweet in api.GetStreamFilter(track=['sports','life','the','a']):
    print(tweet)
    break

### Read tweets from provided JSON

In [180]:
import pandas as pd
import json

# read in file
file = "farmers-protest-tweets.json"
df = pd.read_csv(file,sep='\\n', names=['json_str'], header=None)#, nrows=100)

# translate json str to json struct
df['parsed'] = df['json_str'].apply(json.loads)

# flatten JSON
df_norm = pd.json_normalize(df['parsed'])

# Clean columns
df_parsed = df.drop(columns=['json_str', 'parsed']).join(df_norm)
print(df_parsed.columns)

/var/folders/cj/yhf6q8bn0nqfvv1_wrdfq7gw0000gn/T/ipykernel_46082/1613217607.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file,sep='\\n', names=['json_str'], header=None)#, nrows=100)


Index(['url', 'date', 'content', 'renderedContent', 'id', 'outlinks',
       'tcooutlinks', 'replyCount', 'retweetCount', 'likeCount',
       ...
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quoteCount',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.conversationId',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.lang',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.source',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.sourceUrl',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.sourceLabel',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.media',
       'quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.quotedTweet.retweetedTweet',
       'quotedTweet.q

In [204]:
df_content = df_parsed[['content']].copy()
print(df_content.head(5).to_markdown())

|    | content                                                                                                         |
|---:|:----------------------------------------------------------------------------------------------------------------|
|  0 | Support 👇                                                                                                      |
|    |                                                                                                                 |
|    | #FarmersProtest                                                                                                 |
|  1 | Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest https://t.co/iWPatTK6NV |
|  2 | Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest https://t.co/TcoZx4ZmyU      |
|  3 | #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.         |
|  4 | You hate farmers I hate yo

### Clean Tweets

#### abbreviations

In [205]:
# source: https://www.kaggle.com/code/nmaguette/up-to-date-list-of-slangs-for-text-preprocessing
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "gud" : "good",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

#### text processing

In [206]:
import re
from html import unescape

n = 5

# remove html syntax
df_content["unescape"] = df_content["content"].apply(unescape)
print(df_content["unescape"].head(n).to_markdown(index=False),end='\n\n')

def replace_newlines(text):
    return re.sub(r'\s*\n\s*', ' ', text).strip()
df_content["newlines"] = df_content["unescape"].apply(replace_newlines)
print(df_content["newlines"].head(n).to_markdown(index=False),end='\n\n')

# remove urls
def remove_urls(text):
    url_pattern = re.compile(r"http\S+")
    return re.sub(url_pattern, "", text).strip()
df_content['no_urls'] = df_content['newlines'].apply(remove_urls)
print(df_content["no_urls"].head(n).to_markdown(index=False),end='\n\n')

# remove usernames
def replace_usernames(text):
    username_pattern = re.compile(r"@\S+")
    return re.sub(username_pattern, "", text).strip()
df_content['no_uname'] = df_content['no_urls'].apply(replace_usernames)
print(df_content["no_uname"].head(n).to_markdown(index=False),end='\n\n')

# replace abbreviations
def replace_abbrev(text):
    return ' '.join([abbreviations.get(word.lower(), word) for word in text.split()]).strip()
df_content['rep_abbrev'] = df_content['no_uname'].apply(replace_abbrev)
print(df_content["rep_abbrev"].head(n).to_markdown(index=False),end='\n\n')

# remove elongation
def reduce_elongation(text):
    return re.sub(r'(.)\1{2,}', r'\1', text).strip()
df_content['rem_elong'] = df_content['rep_abbrev'].apply(reduce_elongation)
print(df_content["rem_elong"].head(n).to_markdown(index=False),end='\n\n')



| unescape                                                                                                        |
|:----------------------------------------------------------------------------------------------------------------|
| Support 👇                                                                                                      |
|                                                                                                                 |
| #FarmersProtest                                                                                                 |
| Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest https://t.co/iWPatTK6NV |
| Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest https://t.co/TcoZx4ZmyU      |
| #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand.         |
| You hate farmers I hate you,                                           

In [209]:
import string
import contractions as contr
import emoji


# remove emojis
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')
df_content['remove_emojis'] = df_content['rem_elong'].apply(remove_emojis)
print(df_content["remove_emojis"].head(n).to_markdown(index=False),end='\n\n')

# remove contractions
df_content['no_cont'] = df_content['remove_emojis'].apply(contr.fix)
print(df_content["no_cont"].head(n).to_markdown(index=False),end='\n\n')

# remove punctuation - leave in ! and ? per instructions
def remove_punctuation(text):
    exclude = string.punctuation.replace('!', '').replace('?', '').replace('#', '')
    translator = str.maketrans('', '', exclude)
    return text.translate(translator)
df_content['no_punc'] = df_content['no_cont'].apply(remove_punctuation)
print(df_content["no_punc"].head(n).to_markdown(index=False),end='\n\n')

# remove hashtags
def remove_hashtags(text):
    hashtag_pattern = re.compile(r"#\S+")
    return re.sub(hashtag_pattern, "", text).strip()
df_content['no_tags'] = df_content['no_punc'].apply(remove_hashtags)
print(df_content["no_tags"].head(n).to_markdown(index=False),end='\n\n')

# remove redundant space - should be last
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()
df_content['clean'] = df_content['no_tags'].apply(normalize_whitespace)
print(df_content["clean"].head(n).to_markdown(index=False),end='\n\n')

| remove_emojis                                                                                           |
|:--------------------------------------------------------------------------------------------------------|
| Support  #FarmersProtest                                                                                |
| Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest                 |
| Support farmers if you are related to food #StopHateAgainstFarmers #FarmersProtest                      |
| #StopHateAgainstFarmers support #FarmersProtest by pressurising got. to accept farmer's genuine demand. |
| You hate farmers I hate you, if you love them I love you! ;) #StopHateAgainstFarmers #FarmersProtest    |

| no_cont                                                                                                 |
|:--------------------------------------------------------------------------------------------------------|
| Support  #FarmersProtest 

#### pipeline preview

In [215]:
def run_nlp_pipeline(text, debug=False):
    steps = [
        ("Original", lambda x: x),
        ("Unescaped", unescape),
        ("Single Line", replace_newlines),
        ("No URLs", remove_urls),
        ("Usernames replaced", replace_usernames),
        ("Abbreviations replaced", replace_abbrev),
        ("Reduced elongation", reduce_elongation),
        ("Remove Emojis", remove_emojis),
        ("Expanded contractions", contr.fix),
        ("No punctuation", remove_punctuation),
        ("No hashtags", remove_hashtags),
        ("Normalized whitespace", normalize_whitespace)
    ]

    if debug: max_len = max(len(name) for name, _ in steps)

    for name, func in steps:
        text = func(text)
        if debug: print(f"{name.ljust(max_len)} : {text}")

    return text

test = "\n@user_@34 Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like http://lifeisgreat.com ."
run_nlp_pipeline(test, debug=True)

Original               : 
@user_@34 Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like http://lifeisgreat.com .
Unescaped              : 
@user_@34 Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like http://lifeisgreat.com .
Single Line            : @user_@34 Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like http://lifeisgreat.com .
No URLs                : @user_@34 Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like  .
Usernames replaced     : Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like  .
Abbreviations replaced : Life is great 👇 & I like it sooooooooo much! It's whatis life. #life #great#like .
Reduced elongation     : Life is great 👇 & I like it so much! It's whatis life. #life #great#like .
Remove Emojis          : Life is great  & I like it so much! It's whatis life. #life #great#like .
Expanded contractions  : Life is great  

'Life is great I like it so much! It is whatis life'

#### Calculate Polarity

In [217]:
from nltk.sentiment import SentimentIntensityAnalyzer

def analyze_polarity(text, analyzer=SentimentIntensityAnalyzer()):
    score = analyzer.polarity_scores(text)['compound']
    # print(f"\npolarity score: {score} - {"positive" if score>=0 else "negative"}")
    classification = "positive" if score>=0 else "negative"
    return (score, classification)

# apply full pipeline
df_results = df_parsed[["content"]].copy()
df_results["cleaned"]  = df_results["content"].apply(run_nlp_pipeline)
print(df_results.head(n).to_markdown(index=False),end='\n\n')

# calculate polarity
df_results["polarity"] = df_results["cleaned"].apply(analyze_polarity)
print(df_results[["cleaned", "polarity"]].head(n).to_markdown(index=False),end='\n\n')

| content                                                                                                         | cleaned                                                      |
|:----------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------|
| Support 👇                                                                                                      | Support                                                      |
|                                                                                                                 |                                                              |
| #FarmersProtest                                                                                                 |                                                              |
| Supporting farmers means supporting our country #StopHateAgainstFarmers #FarmersProtest https://t.co/iWP